In [1]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
import warnings
from datetime import datetime

# 불필요한 경고 메시지를 숨깁니다.
warnings.filterwarnings('ignore')

In [2]:
def predict(lotto_history_data, n_estimators=100, random_state=350, verbose=0):
    # 데이터를 Pandas DataFrame으로 변환합니다.
    df = pd.DataFrame(lotto_history_data, columns=['회차', '번호1', '번호2', '번호3', '번호4', '번호5', '번호6'])
    df = df.sort_values(by='회차').reset_index(drop=True)

    if verbose > 0:
        print("--- 로또 당첨 번호 이력 ---")
        print(df)

    # 기계 학습을 위한 데이터 준비
    # 각 회차의 당첨 번호(X)와 바로 다음 회차의 당첨 번호(y)를 사용합니다.
    X = df[['번호1', '번호2', '번호3', '번호4', '번호5', '번호6']]
    y = df[['번호1', '번호2', '번호3', '번호4', '번호5', '번호6']].shift(-1)

    # 마지막 행은 다음 회차 데이터가 없으므로 제거합니다.
    X = X[:-1]
    y = y.dropna()
    
    if verbose > 0:
        print("\n--- 훈련 데이터 (X) ---")
        print(X.head())
        print("\n--- 정답 데이터 (y) ---")
        print(y.head())

    # 머신러닝 모델 선택 및 훈련
    # 랜덤 포레스트 회귀 모델을 사용합니다.
    # the random_state parameter is used to control the randomness of the algorithm, ensuring reproducibility of results. 
    # the n_estimators parameter specifies the number of decision trees in the forest. 
    # model = RandomForestRegressor(n_estimators=50, random_state=1000) # n_estimators: 만들 트리의 개수 => 3개 일치
    model = RandomForestRegressor(n_estimators=n_estimators, random_state=random_state) # n_estimators: 만들 트리의 개수
    model.fit(X, y)

    # 예측할 회차의 이전 회차 데이터 (1173회차)
    last_draw = df.iloc[-1][['번호1', '번호2', '번호3', '번호4', '번호5', '번호6']].values.reshape(1, -1)

    if verbose > 0:
        print(f"\n--- {df.iloc[-1]['회차']}회차 데이터로 다음 회차 예측 ---")
        print(last_draw)

    # 다음 회차(1174회) 번호 예측
    predicted_numbers_set = []
    trial = 5
    for i in range(trial):
        predicted_numbers_float = model.predict(last_draw)

        # 예측된 번호 처리
        # 1. 소수점을 반올림하여 정수로 만듭니다.
        # 2. 1~45 사이의 값으로 보정합니다.
        # 3. 중복된 번호를 제거하고 6개를 선택합니다.
        predicted_numbers = set()
        for num in predicted_numbers_float[0]:
            # 반올림하여 정수로 변환
            int_num = int(round(num))
    
            # 1보다 작으면 1로, 45보다 크면 45로 보정
            if int_num < 1:
                int_num = 1
            elif int_num > 45:
                int_num = 45
            predicted_numbers.add(int_num)

        # 중복 제거 후 6개가 안되면, 부족한 만큼 다른 번호로 채웁니다.
        # (여기서는 가장 빈도가 높은 번호들 중 예측되지 않은 번호를 추가하는 방식을 사용)
        if len(predicted_numbers) < 6:
            all_numbers = df[['번호1', '번호2', '번호3', '번호4', '번호5', '번호6']].values.flatten()
            counts = pd.Series(all_numbers).value_counts()
    
            extra_needed = 6 - len(predicted_numbers)
            for num in counts.index:
                if extra_needed == 0:
                    break
                if num not in predicted_numbers:
                    predicted_numbers.add(num)
                    extra_needed -= 1
        # 최종 예측 번호를 정렬하여 출력
        final_prediction = sorted(list(predicted_numbers))[:6]
        predicted_numbers_set.append(final_prediction)
    return predicted_numbers_set


def test_prediction(round_nums,
                    actual_numbers,
                    bonus,
                    predicted_numbers_set,
                    n_estimators=500,
                    random_state=500,
                    only_one=True,
                    verbose=0):
    if verbose>0:
        print("======================================")
        print(f"\n--- 참고: {round_nums}회 실제 당첨 번호 ---")
        print(f"당첨번호: {actual_numbers}, 보너스: {bonus}")

    matched_len = 0

    for final_prediction in predicted_numbers_set:
        # 예측 결과와 실제 결과 비교
        matching_numbers = set(final_prediction).intersection(set(actual_numbers))
        if verbose > 0:
            print(f"\n--- 예측과 실제 결과 비교 ---")
            print(f"일치하는 번호: {sorted(list(matching_numbers))}")
            print(f"일치 개수: {len(matching_numbers)}")
            print(F"예측 번호: {final_prediction}")
            print(f"n_estimators: {n_estimators}")
            print(f"random_state: {random_state}")
        matched_len = len(matching_numbers)
        if only_one:
            break
    return matched_len

In [7]:
# 실제 1174회 당첨 번호
actual_numbers = [
    [1, 2, 3, 4, 5, 6],
    [8, 11, 14, 17, 36, 39],
    [1, 5, 18, 20, 30, 35],
    [7, 9, 24, 40, 42, 44],
]
bonus = [0, 22, 22, 22]
round_nums = [1175, 1174, 1173, 1172]
# 역대 로또 당첨 번호 데이터 (최신 회차부터 순서대로)
# 실제로는 더 많은 데이터가 필요하지만, 예시를 위해 일부만 사용합니다.
# 데이터 출처: https://data.soledot.com/lottowinnumber/fo/lottowinnumberlist.sd
lotto_history_data = [
    [1174, 8, 11, 14, 17, 36, 39],
    [1173, 1, 5, 18, 20, 30, 35],
    [1172, 7, 9, 24, 40, 42, 44],
    [1171, 3, 6, 7, 11, 12, 17],
    [1170, 3, 13, 28, 34, 38, 42],
    [1169, 5, 12, 24, 26, 39, 42],
    [1168, 9, 21, 24, 30, 33, 37],
    [1167, 8, 23, 31, 35, 39, 40],
    [1166, 14, 23, 25, 27, 29, 42],
    [1165, 6, 7, 27, 29, 38, 45],
    [1164, 17, 18, 23, 25, 38, 39],
    [1163, 2, 13, 15, 16, 33, 43],
    [1162, 20, 21, 22, 25, 28, 29],
    [1161, 2, 12, 20, 24, 34, 42],
    [1160, 7, 13, 18, 36, 39, 45],
    [1159, 3, 9, 27, 28, 38, 39],
    [1158, 21, 25, 27, 32, 37,38],
    [1157, 5, 7, 12, 20, 25, 26],
    [1156, 30, 31, 34, 39, 41, 45],
    [1155, 10, 16, 19, 27, 37, 38],
    [1154, 4, 8, 22, 26, 32, 38],
    [1153, 1, 9, 10, 13, 35, 44],
    [1152, 30, 31, 32, 35, 36, 37],
    [1151, 2, 3, 9, 15, 27, 29],
    [1150, 8, 9, 18, 35, 39, 45],
    [1149, 8, 15, 19, 21, 32, 36],
    [1148, 3, 6, 13, 15, 16, 22],
    [1147, 7, 11, 24, 26, 27, 37],
    [1146, 6, 11, 17, 19, 40, 43],
    [1145, 2, 11, 31, 33, 37, 44],
    [1144, 3, 4, 12, 15, 26, 34],
    [1143, 10, 16, 17, 27, 28, 36],
    [1142, 2, 8, 28, 30, 37, 41],
    [1141, 7, 11, 12, 21, 26, 35],
    [1140, 7, 10, 22, 29, 31, 38],
    [1139, 5, 12, 15, 30, 37, 40],
    [1138, 14, 16, 19, 20, 29, 34],
    [1137, 4, 9, 12, 15, 33, 45],
    [1136, 21, 33, 35, 38, 42, 44],
    [1135, 1, 6, 13, 19, 21, 33],
    [1134, 3, 7, 9, 13, 19, 24],
    [1133, 13, 14, 20, 28, 29, 34],
    [1132, 6, 7, 19, 28, 34, 41],
    [1131, 1, 2, 6, 14, 27, 38],
    [1130, 15, 19, 21, 25, 27, 28],
    # [1129, 5, 10, 11, 17, 28, 34],
]

r_pos = [
    (0, len(lotto_history_data)-2),
    (1, len(lotto_history_data)-1),
    (2, len(lotto_history_data)-0),
    # (3, len(lotto_history_data)-0),
]

results_set=[]
data_poses = [1, 2]

**Notice**
- the random_state parameter is used to control the randomness of the algorithm, ensuring reproducibility of results. 
- the n_estimators parameter specifies the number of decision trees in the forest. This parameter is crucial as it directly impacts the model's performance and computational cost.

In [4]:
def predit_and_test_parts(data_pos, h_data, minus_value, n_estimators, random_state, verbose=0):
    predicted_numbers_set = predict(lotto_history_data=h_data,
                                    n_estimators=n_estimators,
                                    random_state=random_state,
                                    verbose=verbose)
    predict_len1=test_prediction(round_nums[data_pos],
                                 actual_numbers[data_pos],
                                 bonus[data_pos],
                                 predicted_numbers_set,
                                 n_estimators,
                                 random_state,
                                 verbose)
    return predicted_numbers_set, predict_len1


def predict_and_test(minus_value, n_estimators, random_state, verbose=0):
    predict_lens = []
    for data_pos in data_poses:
        h_data=lotto_history_data[r_pos[data_pos][0]:r_pos[data_pos][1]-minus_value]
        _, predict_len = predit_and_test_parts(data_pos, h_data, minus_value, n_estimators, random_state, 0)
        predict_lens.append(predict_len)
    return predict_lens

In [5]:
results_set = []

In [19]:
n_estimators=25
begin_pos=0
end_pos=600000
print(f'start   [now={datetime.now()}]')
for minus_value in [8, 7, 6, 5, 4, 3, 2, 1]:
    cnt = 0
    for random_state in range(begin_pos, end_pos, 100):
        predict_lens = predict_and_test(minus_value, n_estimators, random_state, 0)
        if predict_lens[0] > 2 and predict_lens[1] > 2:
            results_set.append((n_estimators, random_state, predict_lens, minus_value))
            print('found = ', n_estimators, random_state, predict_lens, minus_value)
        cnt += 1
        if cnt % 1000 == 0:
            print(f'proceed [now={datetime.now()}, {cnt}]')        
print(f'completed [now={datetime.now()}]')

start   [now=2025-06-06 21:19:57.866841]
found =  25 38300 [3, 3] 0
found =  25 71900 [3, 3] 0
found =  25 72500 [3, 3] 0
found =  25 96600 [3, 3] 0
proceed [now=2025-06-06 21:20:45.855791, 1000]
proceed [now=2025-06-06 21:21:33.774185, 2000]
found =  25 208900 [3, 3] 0
found =  25 219100 [3, 4] 0
proceed [now=2025-06-06 21:22:21.758462, 3000]
proceed [now=2025-06-06 21:23:09.635637, 4000]
found =  25 434400 [3, 3] 0
found =  25 456800 [3, 3] 0
found =  25 473800 [3, 3] 0
found =  25 480700 [3, 3] 0
proceed [now=2025-06-06 21:23:57.617372, 5000]
found =  25 527700 [3, 3] 0
found =  25 540500 [3, 3] 0
found =  25 560100 [3, 4] 0
proceed [now=2025-06-06 21:24:45.872973, 6000]
proceed [now=2025-06-06 21:25:33.969216, 1000]
found =  25 101100 [3, 3] 1
found =  25 122600 [3, 3] 1
found =  25 195700 [3, 3] 1
proceed [now=2025-06-06 21:26:22.011288, 2000]
found =  25 275200 [3, 3] 1
found =  25 279000 [3, 3] 1
found =  25 282400 [3, 3] 1
proceed [now=2025-06-06 21:27:09.995860, 3000]
found = 

In [20]:
for result in results_set:
    if result[2][0] > 3 or result[2][1] > 3:
        print(result)

(25, 219100, [3, 4], 0)
(25, 560100, [3, 4], 0)
(25, 322300, [3, 4], 5)
(25, 520500, [3, 4], 5)
(25, 148700, [3, 4], 6)
(25, 283500, [3, 4], 6)
(25, 358100, [3, 4], 6)
(25, 451600, [4, 3], 6)
(25, 489000, [4, 3], 6)
(25, 295500, [3, 4], 7)
(25, 241500, [3, 4], 8)


In [26]:
n_estimators=45
begin_pos=0
end_pos=600000
print(f'start   [now={datetime.now()}]')
for minus_value in [8, 7, 6, 5, 4, 3, 2, 1]:
    cnt = 0
    for random_state in range(begin_pos, end_pos, 100):
        predict_lens = predict_and_test(minus_value, n_estimators, random_state, 0)
        if predict_lens[0] > 2 and predict_lens[1] > 2:
            results_set.append((n_estimators, random_state, predict_lens, minus_value))
            print('found = ', n_estimators, random_state, predict_lens, minus_value)
        cnt += 1
        if cnt % 1000 == 0:
            print(f'proceed [now={datetime.now()}, {cnt}]')        
print(f'completed [now={datetime.now()}]')

start   [now=2025-06-06 22:08:39.836185]
proceed [now=2025-06-06 22:09:59.606574, 1000]
proceed [now=2025-06-06 22:11:20.448763, 2000]
proceed [now=2025-06-06 22:12:41.488266, 3000]
proceed [now=2025-06-06 22:14:03.404580, 4000]
proceed [now=2025-06-06 22:15:24.629190, 5000]
proceed [now=2025-06-06 22:16:48.033360, 6000]
proceed [now=2025-06-06 22:18:08.090793, 1000]
proceed [now=2025-06-06 22:19:29.121877, 2000]
found =  45 238300 [3, 3] 7
proceed [now=2025-06-06 22:20:50.422847, 3000]
proceed [now=2025-06-06 22:22:13.143242, 4000]
proceed [now=2025-06-06 22:23:33.839418, 5000]
found =  45 574600 [3, 3] 7
proceed [now=2025-06-06 22:24:56.234796, 6000]
found =  45 11200 [3, 3] 6
found =  45 30300 [3, 3] 6
found =  45 52100 [3, 4] 6
found =  45 58400 [3, 3] 6
found =  45 76900 [3, 3] 6
found =  45 77600 [3, 3] 6
proceed [now=2025-06-06 22:26:19.733471, 1000]
found =  45 100300 [3, 3] 6
found =  45 116800 [3, 3] 6
found =  45 170300 [3, 3] 6
found =  45 176300 [3, 3] 6
proceed [now=2025-

In [57]:
n_estimators=6
begin_pos=100000
end_pos=500000
print(f'start   [now={datetime.now()}]')
for minus_value in [8, 7, 6, 5, 4, 3, 2, 1]:
    cnt = 0
    for random_state in range(begin_pos, end_pos, 100):
        predict_lens = predict_and_test(minus_value, n_estimators, random_state, 0)
        if predict_lens[0] > 1 and predict_lens[1] > 1 and predict_lens[2] > 1:
            results_set.append((n_estimators, random_state, predict_lens, minus_value))
            print('found = ', n_estimators, random_state, predict_lens, minus_value)
        cnt += 1
        if cnt % 1000 == 0:
            print(f'proceed [now={datetime.now()}, {cnt}]')        
print(f'completed [now={datetime.now()}]')

start   [now=2025-06-07 01:24:57.778933]
found =  6 100000 [2, 2, 2] 8
proceed [now=2025-06-07 01:25:27.865100, 1000]
found =  6 213200 [2, 2, 2] 8
found =  6 221100 [2, 2, 2] 8
proceed [now=2025-06-07 01:25:57.465811, 2000]
found =  6 370300 [2, 2, 2] 8
proceed [now=2025-06-07 01:26:27.156571, 3000]
found =  6 498900 [2, 2, 2] 8
proceed [now=2025-06-07 01:26:56.936260, 4000]
proceed [now=2025-06-07 01:27:26.686838, 1000]
proceed [now=2025-06-07 01:27:56.469690, 2000]
found =  6 371300 [2, 2, 2] 7
found =  6 392400 [2, 2, 2] 7
proceed [now=2025-06-07 01:28:26.296803, 3000]
proceed [now=2025-06-07 01:28:56.076444, 4000]
found =  6 100300 [2, 2, 2] 6
proceed [now=2025-06-07 01:29:25.891071, 1000]
proceed [now=2025-06-07 01:29:55.662046, 2000]
found =  6 316700 [3, 2, 2] 6
proceed [now=2025-06-07 01:30:25.383788, 3000]
found =  6 437000 [2, 2, 2] 6
proceed [now=2025-06-07 01:30:55.295790, 4000]
found =  6 112000 [3, 2, 2] 5
found =  6 117600 [2, 2, 2] 5
found =  6 126400 [2, 2, 2] 5
found

```
(25, 219100, [3, 4], 0)
(25, 560100, [3, 4], 0)
(25, 322300, [3, 4], 5)
(25, 520500, [3, 4], 5)
(25, 148700, [3, 4], 6)
(25, 283500, [3, 4], 6)
(25, 358100, [3, 4], 6)
(25, 451600, [4, 3], 6)
(25, 489000, [4, 3], 6)
(25, 295500, [3, 4], 7)
(25, 241500, [3, 4], 8)
(45, 52100, [3, 4], 6)
(45, 587300, [3, 4], 4)
(10, 102500, [3, 4], 7)
(10, 113700, [4, 4], 6)
(10, 164200, [3, 4], 1)
(10, 225300, [3, 4], 8)
(10, 371400, [3, 4], 8)
(10, 255700, [3, 4], 7)
(10, 15200, [3, 4], 7)
(10, 63300, [3, 4], 6)
(6, 131400, [3, 4], 5)
(6, 195200, [4, 3], 5)
(6, 406100, [4, 3], 8)
(6, 325200, [4, 3], 7)
(6, 353200, [4, 3], 7)
(6, 473900, [4, 4], 6)
(6, 363000, [4, 3], 5)
[
(25, 451600, [4, 3], 6),
(25, 489000, [4, 3], 6),
(6, 473900, [4, 4], 6),
(10, 113700, [4, 4], 6),
(10, 15200, [3, 4], 7),
]

```

In [39]:
for result in results_set:
    if result[2][0] > 3 or result[2][1] > 3:
        print(result)

(45, 52100, [3, 4], 6)
(45, 587300, [3, 4], 4)
(10, 102500, [3, 4], 7)
(10, 113700, [4, 4], 6)
(10, 164200, [3, 4], 1)
(10, 225300, [3, 4], 8)
(10, 371400, [3, 4], 8)
(10, 255700, [3, 4], 7)
(10, 15200, [3, 4], 7)
(10, 63300, [3, 4], 6)
(6, 131400, [3, 4], 5)
(6, 195200, [4, 3], 5)
(6, 406100, [4, 3], 8)
(6, 325200, [4, 3], 7)
(6, 353200, [4, 3], 7)
(6, 473900, [4, 4], 6)
(6, 363000, [4, 3], 5)


In [8]:
datas = [
    [45,208000,4,3,5],
    [45,101700,4,3,2],
    [45,101700,4,3,2],
    [45,185400,4,3,2],
    [45,96800,4,3,1],
    [45,58600,3,3,0],
    ]
datas = [
    (25, 451600, [4, 3], 6),
    (25, 489000, [4, 3], 6),
    (6, 473900, [4, 4], 6),
    (10, 113700, [4, 4], 6),
    (10, 15200, [3, 4], 7),
    ]
for data in datas:
    n_estimators=data[0]
    random_state=data[1]
    minus_value=data[3]
    results=[]
    print(f'start [now={datetime.now()}]')
    data_pos=1
    h_data=lotto_history_data[r_pos[data_pos][0]:r_pos[data_pos][1]-minus_value]
    predicted_numbers_set = predict(lotto_history_data=h_data,
                                    n_estimators=n_estimators,
                                    random_state=random_state,
                                    verbose=0)
    predict_len1=test_prediction(round_nums[data_pos],
                                 actual_numbers[data_pos],
                                 bonus[data_pos],
                                 predicted_numbers_set,
                                 n_estimators,
                                 random_state,
                                 verbose=1)
    data_pos=2
    h_data=lotto_history_data[r_pos[data_pos][0]:r_pos[data_pos][1]-minus_value]
    predicted_numbers_set = predict(lotto_history_data=h_data,
                                    n_estimators=n_estimators,
                                    random_state=random_state,
                                    verbose=0)
    predict_len2 = test_prediction(round_nums[data_pos],
                                   actual_numbers[data_pos],
                                   bonus[data_pos],
                                   predicted_numbers_set,
                                   n_estimators,
                                   random_state,
                                   verbose=1)
print(f'completed [now={datetime.now()}]')

start [now=2025-06-07 01:50:46.951576]

--- 참고: 1174회 실제 당첨 번호 ---
당첨번호: [8, 11, 14, 17, 36, 39], 보너스: 22

--- 예측과 실제 결과 비교 ---
일치하는 번호: [11, 17, 36, 39]
일치 개수: 4
예측 번호: [11, 17, 23, 30, 36, 39]
n_estimators: 25
random_state: 451600

--- 참고: 1173회 실제 당첨 번호 ---
당첨번호: [1, 5, 18, 20, 30, 35], 보너스: 22

--- 예측과 실제 결과 비교 ---
일치하는 번호: [5, 30, 35]
일치 개수: 3
예측 번호: [5, 10, 15, 21, 30, 35]
n_estimators: 25
random_state: 451600
start [now=2025-06-07 01:50:47.008282]

--- 참고: 1174회 실제 당첨 번호 ---
당첨번호: [8, 11, 14, 17, 36, 39], 보너스: 22

--- 예측과 실제 결과 비교 ---
일치하는 번호: [11, 17, 36, 39]
일치 개수: 4
예측 번호: [11, 17, 25, 30, 36, 39]
n_estimators: 25
random_state: 489000

--- 참고: 1173회 실제 당첨 번호 ---
당첨번호: [1, 5, 18, 20, 30, 35], 보너스: 22

--- 예측과 실제 결과 비교 ---
일치하는 번호: [5, 20, 30]
일치 개수: 3
예측 번호: [5, 10, 15, 20, 30, 36]
n_estimators: 25
random_state: 489000
start [now=2025-06-07 01:50:47.057962]

--- 참고: 1174회 실제 당첨 번호 ---
당첨번호: [8, 11, 14, 17, 36, 39], 보너스: 22

--- 예측과 실제 결과 비교 ---
일치하는 번호: [11, 17, 36, 39]
일치 개수:

In [9]:
# datas = [
#     [45,208000,4,3,5],
#     [45,101700,4,3,2],
#     [45,185400,4,3,2],
#     [45,96800,4,3,1],
#     [45,58600,3,3,0],
#     ]
datas = [
    (25, 451600, [4, 3], 6),
    (25, 489000, [4, 3], 6),
    (6, 473900, [4, 4], 6),
    (10, 113700, [4, 4], 6),
    (10, 15200, [3, 4], 7),
    ]
print(f'start [now={datetime.now()}]')
results=[]
for data in datas:
    n_estimators=data[0]
    random_state=data[1]
    minus_value=data[3]    
    data_pos=0
    h_data=lotto_history_data[r_pos[data_pos][0]:r_pos[data_pos][1]-minus_value]
    predicted_numbers_set = predict(lotto_history_data=h_data,
                                    n_estimators=n_estimators,
                                    random_state=random_state,
                                    verbose=0)
    results.append(predicted_numbers_set[0])
    predict_len2 = test_prediction(round_nums[data_pos],
                                   actual_numbers[1],
                                   bonus[data_pos],
                                   predicted_numbers_set,
                                   n_estimators,
                                   random_state,
                                   verbose=1)
print(f'completed [now={datetime.now()}]')
print('Predicted Numbers.')
for result in results:
    print(result)

start [now=2025-06-07 01:51:52.939283]

--- 참고: 1175회 실제 당첨 번호 ---
당첨번호: [8, 11, 14, 17, 36, 39], 보너스: 0

--- 예측과 실제 결과 비교 ---
일치하는 번호: [8]
일치 개수: 1
예측 번호: [8, 12, 22, 29, 33, 40]
n_estimators: 25
random_state: 451600

--- 참고: 1175회 실제 당첨 번호 ---
당첨번호: [8, 11, 14, 17, 36, 39], 보너스: 0

--- 예측과 실제 결과 비교 ---
일치하는 번호: []
일치 개수: 0
예측 번호: [9, 12, 21, 27, 32, 38]
n_estimators: 25
random_state: 489000

--- 참고: 1175회 실제 당첨 번호 ---
당첨번호: [8, 11, 14, 17, 36, 39], 보너스: 0

--- 예측과 실제 결과 비교 ---
일치하는 번호: [11, 36]
일치 개수: 2
예측 번호: [5, 11, 21, 30, 36, 42]
n_estimators: 6
random_state: 473900

--- 참고: 1175회 실제 당첨 번호 ---
당첨번호: [8, 11, 14, 17, 36, 39], 보너스: 0

--- 예측과 실제 결과 비교 ---
일치하는 번호: [39]
일치 개수: 1
예측 번호: [7, 10, 20, 28, 33, 39]
n_estimators: 10
random_state: 113700

--- 참고: 1175회 실제 당첨 번호 ---
당첨번호: [8, 11, 14, 17, 36, 39], 보너스: 0

--- 예측과 실제 결과 비교 ---
일치하는 번호: [36]
일치 개수: 1
예측 번호: [9, 13, 21, 28, 31, 36]
n_estimators: 10
random_state: 15200
completed [now=2025-06-07 01:51:53.034252]
Predicted Numbers.


In [ ]:
print(results)

```
[
[45,208000,4,3,5],
[45,101700,4,3,2],
[45,101700,4,3,2],
[45,185400,4,3,2],
[45,96800,4,3,1],
    [45,58600,3,3,0],
]
```

In [ ]:
n_estimators=10
random_state=5000
data_pos=1
h_data=lotto_history_data[r_pos[data_pos][0]:r_pos[data_pos][1]]
predicted_numbers_set = predict(lotto_history_data=h_data,
                                n_estimators=n_estimators,
                                random_state=random_state,
                                verbose=0
                               )
test_prediction(round_nums[data_pos],
                actual_numbers[data_pos],
                bonus[data_pos],
                predicted_numbers_set,
                n_estimators,
                random_state)
data_pos=2
h_data=lotto_history_data[r_pos[data_pos][0]:r_pos[data_pos][1]]
predicted_numbers_set = predict(lotto_history_data=h_data,
                                n_estimators=n_estimators,
                                random_state=random_state,
                                verbose=0)
test_prediction(round_nums[data_pos],
                actual_numbers[data_pos],
                bonus[data_pos],
                predicted_numbers_set,
                n_estimators,
                random_state)

In [ ]:
[
    ([14, 17, 23, 27, 34, 36], [14, 17, 36], (45, 1000), (1160, 1173),
    ([13, 16, 23, 27, 34, 36], [36], (45, 1300), (1160, 1173)),
    ([11, 15, 21, 30, 36, 39], [11, 36, 39], (45, 1000), (1160, 1173),

]